In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import glob
from tqdm import tqdm
from music21 import *
from tensorflow.python.ops import control_flow_ops
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
notes = []
for file in glob.glob("*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [4]:
n_vocab=len(set(notes))

In [5]:
sequence_length = 500
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [ ]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(256))
model.add(Dense(n_vocab,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['categorical_crossentropy','accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 500, 256)          264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 80)                20560     
Total params: 810,064
Trainable params: 810,064
Non-trainable params: 0
_________________________________________________________________


In [ ]:
filepath = ".\\Weights\\weights-improvement-{epoch}-singleMidi.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
history = model.fit(network_input, network_output, epochs=500, batch_size=100, callbacks=callbacks_list)
plot_training_curves(history.history);

Epoch 1/500
429/429 [==============================] - ETA: 30s - loss: 4.3816 - categorical_crossentropy: 4.3816 - acc: 0.030 - ETA: 20s - loss: 4.3163 - categorical_crossentropy: 4.3163 - acc: 0.125 - ETA: 11s - loss: 4.2641 - categorical_crossentropy: 4.2641 - acc: 0.133 - ETA: 2s - loss: 4.2072 - categorical_crossentropy: 4.2072 - acc: 0.110 - 40s - loss: 4.1749 - categorical_crossentropy: 4.1749 - acc: 0.1096    
Epoch 2/500
429/429 [==============================] - ETA: 30s - loss: 3.4758 - categorical_crossentropy: 3.4758 - acc: 0.100 - ETA: 21s - loss: 3.5548 - categorical_crossentropy: 3.5548 - acc: 0.065 - ETA: 12s - loss: 3.5389 - categorical_crossentropy: 3.5389 - acc: 0.090 - ETA: 2s - loss: 3.5307 - categorical_crossentropy: 3.5307 - acc: 0.110 - 42s - loss: 3.5355 - categorical_crossentropy: 3.5355 - acc: 0.1142    
Epoch 3/500
429/429 [==============================] - ETA: 32s - loss: 3.5893 - categorical_crossentropy: 3.5893 - acc: 0.140 - ETA: 22s - loss: 3.5088 - c

429/429 [==============================] - ETA: 42s - loss: 3.0897 - categorical_crossentropy: 3.0897 - acc: 0.170 - ETA: 29s - loss: 3.0678 - categorical_crossentropy: 3.0678 - acc: 0.175 - ETA: 16s - loss: 3.0493 - categorical_crossentropy: 3.0493 - acc: 0.173 - ETA: 3s - loss: 3.0625 - categorical_crossentropy: 3.0625 - acc: 0.175 - 58s - loss: 3.0676 - categorical_crossentropy: 3.0676 - acc: 0.1772    
Epoch 21/500
429/429 [==============================] - ETA: 44s - loss: 3.0520 - categorical_crossentropy: 3.0520 - acc: 0.180 - ETA: 31s - loss: 2.9451 - categorical_crossentropy: 2.9451 - acc: 0.210 - ETA: 17s - loss: 3.0286 - categorical_crossentropy: 3.0286 - acc: 0.186 - ETA: 4s - loss: 3.0521 - categorical_crossentropy: 3.0521 - acc: 0.187 - 60s - loss: 3.0586 - categorical_crossentropy: 3.0586 - acc: 0.1888    
Epoch 22/500
429/429 [==============================] - ETA: 46s - loss: 3.4560 - categorical_crossentropy: 3.4560 - acc: 0.080 - ETA: 33s - loss: 3.3420 - categorical

429/429 [==============================] - ETA: 56s - loss: 2.7342 - categorical_crossentropy: 2.7342 - acc: 0.280 - ETA: 40s - loss: 2.6871 - categorical_crossentropy: 2.6871 - acc: 0.240 - ETA: 22s - loss: 2.5961 - categorical_crossentropy: 2.5961 - acc: 0.270 - ETA: 5s - loss: 2.6534 - categorical_crossentropy: 2.6534 - acc: 0.245 - 76s - loss: 2.6714 - categorical_crossentropy: 2.6714 - acc: 0.2448    
Epoch 40/500
429/429 [==============================] - ETA: 55s - loss: 2.6127 - categorical_crossentropy: 2.6127 - acc: 0.250 - ETA: 38s - loss: 2.5948 - categorical_crossentropy: 2.5948 - acc: 0.255 - ETA: 22s - loss: 2.6112 - categorical_crossentropy: 2.6112 - acc: 0.236 - ETA: 5s - loss: 2.6010 - categorical_crossentropy: 2.6010 - acc: 0.247 - 76s - loss: 2.6292 - categorical_crossentropy: 2.6292 - acc: 0.2494    
Epoch 41/500
429/429 [==============================] - ETA: 56s - loss: 2.7176 - categorical_crossentropy: 2.7176 - acc: 0.220 - ETA: 40s - loss: 2.6716 - categorical

429/429 [==============================] - ETA: 60s - loss: 2.2224 - categorical_crossentropy: 2.2224 - acc: 0.280 - ETA: 42s - loss: 2.4599 - categorical_crossentropy: 2.4599 - acc: 0.255 - ETA: 24s - loss: 2.6210 - categorical_crossentropy: 2.6210 - acc: 0.243 - ETA: 5s - loss: 2.5495 - categorical_crossentropy: 2.5495 - acc: 0.240 - 80s - loss: 2.5245 - categorical_crossentropy: 2.5245 - acc: 0.2448    
Epoch 59/500
429/429 [==============================] - ETA: 58s - loss: 2.1514 - categorical_crossentropy: 2.1514 - acc: 0.290 - ETA: 41s - loss: 2.1580 - categorical_crossentropy: 2.1580 - acc: 0.310 - ETA: 23s - loss: 2.1822 - categorical_crossentropy: 2.1822 - acc: 0.306 - ETA: 5s - loss: 2.1836 - categorical_crossentropy: 2.1836 - acc: 0.307 - 79s - loss: 2.1834 - categorical_crossentropy: 2.1834 - acc: 0.3030    
Epoch 60/500
429/429 [==============================] - ETA: 59s - loss: 2.2862 - categorical_crossentropy: 2.2862 - acc: 0.280 - ETA: 41s - loss: 2.2448 - categorical

429/429 [==============================] - ETA: 61s - loss: 1.8936 - categorical_crossentropy: 1.8936 - acc: 0.360 - ETA: 43s - loss: 1.9406 - categorical_crossentropy: 1.9406 - acc: 0.350 - ETA: 24s - loss: 1.9956 - categorical_crossentropy: 1.9956 - acc: 0.346 - ETA: 5s - loss: 2.0244 - categorical_crossentropy: 2.0244 - acc: 0.342 - 82s - loss: 2.0258 - categorical_crossentropy: 2.0258 - acc: 0.3403    
Epoch 78/500
429/429 [==============================] - ETA: 62s - loss: 2.4364 - categorical_crossentropy: 2.4364 - acc: 0.150 - ETA: 44s - loss: 2.3269 - categorical_crossentropy: 2.3269 - acc: 0.185 - ETA: 24s - loss: 2.2905 - categorical_crossentropy: 2.2905 - acc: 0.230 - ETA: 5s - loss: 2.2615 - categorical_crossentropy: 2.2615 - acc: 0.250 - 82s - loss: 2.2224 - categorical_crossentropy: 2.2224 - acc: 0.2657    
Epoch 79/500
429/429 [==============================] - ETA: 64s - loss: 2.0971 - categorical_crossentropy: 2.0971 - acc: 0.360 - ETA: 43s - loss: 2.0040 - categorical

...

429/429 [==============================] - ETA: 63s - loss: 0.2942 - categorical_crossentropy: 0.2942 - acc: 0.930 - ETA: 45s - loss: 0.2639 - categorical_crossentropy: 0.2639 - acc: 0.945 - ETA: 25s - loss: 0.2770 - categorical_crossentropy: 0.2770 - acc: 0.926 - ETA: 5s - loss: 0.2736 - categorical_crossentropy: 0.2736 - acc: 0.927 - 84s - loss: 0.2753 - categorical_crossentropy: 0.2753 - acc: 0.9207    
Epoch 344/500
429/429 [==============================] - ETA: 64s - loss: 0.2855 - categorical_crossentropy: 0.2855 - acc: 0.920 - ETA: 45s - loss: 0.2827 - categorical_crossentropy: 0.2827 - acc: 0.915 - ETA: 25s - loss: 0.2869 - categorical_crossentropy: 0.2869 - acc: 0.900 - ETA: 5s - loss: 0.2870 - categorical_crossentropy: 0.2870 - acc: 0.900 - 85s - loss: 0.2819 - categorical_crossentropy: 0.2819 - acc: 0.9044    
Epoch 345/500
429/429 [==============================] - ETA: 65s - loss: 0.2457 - categorical_crossentropy: 0.2457 - acc: 0.930 - ETA: 45s - loss: 0.2482 - categoric

429/429 [==============================] - ETA: 67s - loss: 0.2419 - categorical_crossentropy: 0.2419 - acc: 0.950 - ETA: 47s - loss: 0.2360 - categorical_crossentropy: 0.2360 - acc: 0.945 - ETA: 26s - loss: 0.2317 - categorical_crossentropy: 0.2317 - acc: 0.940 - ETA: 6s - loss: 0.2245 - categorical_crossentropy: 0.2245 - acc: 0.940 - 88s - loss: 0.2265 - categorical_crossentropy: 0.2265 - acc: 0.9394    
Epoch 363/500
429/429 [==============================] - ETA: 67s - loss: 0.2800 - categorical_crossentropy: 0.2800 - acc: 0.900 - ETA: 46s - loss: 0.2458 - categorical_crossentropy: 0.2458 - acc: 0.935 - ETA: 26s - loss: 0.2336 - categorical_crossentropy: 0.2336 - acc: 0.936 - ETA: 5s - loss: 0.2356 - categorical_crossentropy: 0.2356 - acc: 0.927 - 86s - loss: 0.2315 - categorical_crossentropy: 0.2315 - acc: 0.9277    
Epoch 364/500
429/429 [==============================] - ETA: 66s - loss: 0.2235 - categorical_crossentropy: 0.2235 - acc: 0.920 - ETA: 46s - loss: 0.1998 - categoric

429/429 [==============================] - ETA: 64s - loss: 0.1211 - categorical_crossentropy: 0.1211 - acc: 1.000 - ETA: 46s - loss: 0.1474 - categorical_crossentropy: 0.1474 - acc: 0.990 - ETA: 26s - loss: 0.1652 - categorical_crossentropy: 0.1652 - acc: 0.973 - ETA: 5s - loss: 0.1846 - categorical_crossentropy: 0.1846 - acc: 0.965 - 87s - loss: 0.1884 - categorical_crossentropy: 0.1884 - acc: 0.9650    
Epoch 382/500
429/429 [==============================] - ETA: 66s - loss: 0.2516 - categorical_crossentropy: 0.2516 - acc: 0.930 - ETA: 46s - loss: 0.2245 - categorical_crossentropy: 0.2245 - acc: 0.945 - ETA: 25s - loss: 0.1933 - categorical_crossentropy: 0.1933 - acc: 0.953 - ETA: 5s - loss: 0.1956 - categorical_crossentropy: 0.1956 - acc: 0.955 - 85s - loss: 0.1904 - categorical_crossentropy: 0.1904 - acc: 0.9534    
Epoch 383/500
429/429 [==============================] - ETA: 64s - loss: 0.1597 - categorical_crossentropy: 0.1597 - acc: 0.980 - ETA: 46s - loss: 0.1536 - categoric

429/429 [==============================] - ETA: 66s - loss: 0.1558 - categorical_crossentropy: 0.1558 - acc: 0.960 - ETA: 45s - loss: 0.1339 - categorical_crossentropy: 0.1339 - acc: 0.965 - ETA: 26s - loss: 0.1321 - categorical_crossentropy: 0.1321 - acc: 0.963 - ETA: 5s - loss: 0.1480 - categorical_crossentropy: 0.1480 - acc: 0.950 - 86s - loss: 0.1473 - categorical_crossentropy: 0.1473 - acc: 0.9487    
Epoch 401/500
429/429 [==============================] - ETA: 66s - loss: 0.3470 - categorical_crossentropy: 0.3470 - acc: 0.910 - ETA: 46s - loss: 0.4055 - categorical_crossentropy: 0.4055 - acc: 0.885 - ETA: 26s - loss: 0.4142 - categorical_crossentropy: 0.4142 - acc: 0.876 - ETA: 5s - loss: 0.3696 - categorical_crossentropy: 0.3696 - acc: 0.885 - 86s - loss: 0.3566 - categorical_crossentropy: 0.3566 - acc: 0.8881    
Epoch 402/500
429/429 [==============================] - ETA: 67s - loss: 0.1601 - categorical_crossentropy: 0.1601 - acc: 0.950 - ETA: 47s - loss: 0.1424 - categoric

429/429 [==============================] - ETA: 68s - loss: 0.0860 - categorical_crossentropy: 0.0860 - acc: 0.970 - ETA: 47s - loss: 0.0842 - categorical_crossentropy: 0.0842 - acc: 0.975 - ETA: 26s - loss: 0.0852 - categorical_crossentropy: 0.0852 - acc: 0.976 - ETA: 5s - loss: 0.0820 - categorical_crossentropy: 0.0820 - acc: 0.982 - 87s - loss: 0.0791 - categorical_crossentropy: 0.0791 - acc: 0.9837    
Epoch 420/500
429/429 [==============================] - ETA: 65s - loss: 0.0494 - categorical_crossentropy: 0.0494 - acc: 0.990 - ETA: 45s - loss: 0.0541 - categorical_crossentropy: 0.0541 - acc: 0.990 - ETA: 26s - loss: 0.0524 - categorical_crossentropy: 0.0524 - acc: 0.993 - ETA: 5s - loss: 0.0635 - categorical_crossentropy: 0.0635 - acc: 0.987 - 87s - loss: 0.0630 - categorical_crossentropy: 0.0630 - acc: 0.9883    
Epoch 421/500
429/429 [==============================] - ETA: 65s - loss: 0.0346 - categorical_crossentropy: 0.0346 - acc: 0.990 - ETA: 45s - loss: 0.0536 - categoric

In [ ]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(256))
model.add(Dense(n_vocab,activation='softmax'))
model.load_weights('.\\Weights\\weights-improvement-49-singleMidi.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['categorical_crossentropy','accuracy'])
model.summary()


In [ ]:
# Load the weights to each node
#model.load_weights('weights-improvement-09-3.2697-bigger.hdf5')

In [ ]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
pattern=list(pattern)
# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')